# This notebook contains our preprocessing for predicting the model

Import necessary libraries

In [38]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import math

Here we read in the statistics against each team that made it to the playoffs for the specific seasons. Oddly, our original preprocessing didn't fix the naming, so we fix it again here. 

In [39]:
'''READS IN TEAM STATS AGAINST'''
opp_stats1314 = pd.read_csv('./preprocessed/opp_stats1314.csv', sep=',')
opp_stats1415 = pd.read_csv('./preprocessed/opp_stats1415.csv', sep=',')
opp_stats1516 = pd.read_csv('./preprocessed/opp_stats1516.csv', sep=',')
opp_stats1617 = pd.read_csv('./preprocessed/opp_stats1617.csv', sep=',')
opp_stats1718 = pd.read_csv('./preprocessed/opp_stats1718.csv', sep=',')

opp_arr = [opp_stats1314, opp_stats1415, opp_stats1516, opp_stats1617, opp_stats1718]

for df in opp_arr:
    df['Team'] = df['Team'].str[:-1]

Here we read in the csv's for the statistics for each team that made it to the playoffs in the correct season. We again have to fix the naming of each team. 

In [40]:
'''READS IN TEAM STATS FOR AND CORRECTS NAMING ERRORS'''
team_stats1314 = pd.read_csv('./preprocessed/team_stats1314.csv', sep=',')
team_stats1415 = pd.read_csv('./preprocessed/team_stats1415.csv', sep=',')
team_stats1516 = pd.read_csv('./preprocessed/team_stats1516.csv', sep=',')
team_stats1617 = pd.read_csv('./preprocessed/team_stats1617.csv', sep=',')
team_stats1718 = pd.read_csv('./preprocessed/team_stats1718.csv', sep=',')

dfs = [team_stats1314, team_stats1415, team_stats1516, team_stats1617, team_stats1718]

for df in dfs:
    df['Team'] = df.Team.str.replace(' ', '_')
    df['Team'] = df['Team'].str[:-1]

team_stats1415.head()

,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
0,0,1,Golden_State_Warriors,82,240.6,41.6,87.0,0.478,10.8,27.0,...,10.4,34.3,44.7,27.4,9.3,6.0,14.5,19.9,110.0,81.7
1,1,4,LA_Clippers,82,240.6,39.4,83.3,0.473,10.1,26.9,...,9.6,33.1,42.6,24.8,7.8,5.0,12.3,21.3,106.7,68.3
2,2,9,Dallas_Mavericks,82,242.4,39.7,85.8,0.463,8.9,25.4,...,10.5,31.8,42.3,22.5,8.1,4.5,13.0,20.0,105.2,61.0
3,3,11,Toronto_Raptors,82,242.1,37.9,83.3,0.455,8.9,25.1,...,10.7,30.8,41.5,20.7,7.5,4.4,12.9,20.9,104.0,59.8
4,4,3,Houston_Rockets,82,241.5,37.0,83.3,0.444,11.4,32.7,...,11.7,32.0,43.7,22.2,9.5,5.0,16.7,22.0,103.9,68.3


Then we read in data for who won specific matchups for each season. If the winner is 0, then team a won and if 1, team b won. 

In [41]:
'''READS IN SERIES INFORMATION CSV FILES'''
series_in1314 = pd.read_csv('./Playoff_Matchups/Playoff_13-14.csv', sep=',')
series_in1415 = pd.read_csv('./Playoff_Matchups/Playoff_14-15.csv', sep=',')
series_in1516 = pd.read_csv('./Playoff_Matchups/Playoff_15-16.csv', sep=',')
series_in1617 = pd.read_csv('./Playoff_Matchups/Playoff_16-17.csv', sep=',')
series_in1617.head()

,TeamA,TeamB,Winner
0,Boston_Celtics,Chicago_Bulls,0
1,Washington_Wizards,Atlanta_Hawks,0
2,Toronto_Raptors,Milwaukee_Bucks,0
3,Cleveland_Cavaliers,Indiana_Pacers,0
4,Golden_State_Warriors,Portland_Trail_Blazers,0


Here we have a method that creates the series for each year. 

In [42]:
def make_series(tm, series, df):   
    if tm == 'a':
        a = series['TeamA'].tolist()
    else:
        a = series['TeamB'].tolist()
            
    team = pd.DataFrame()
    
    for x in a:
        to_concat = df[df['Team'] == x]
        frames = [team, to_concat]
        team = pd.concat(frames)
    
    return team  

Here we call the method and make the series for teams a and teams b for each year. As a check, you can see that the team names align correctly with the series dataframe upon. Notice that is is team a, and it has listed all the teams in a

In [43]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM A'''
a = 'a'
team_a1314 = make_series(a, series_in1314, team_stats1314)
team_a_opp1314 = make_series(a, series_in1314, opp_stats1314)
team_a1415 = make_series(a, series_in1415, team_stats1415)
team_a_opp1415 = make_series(a, series_in1415, opp_stats1415)
team_a1516 = make_series(a, series_in1516, team_stats1516)
team_a_opp1516 = make_series(a, series_in1516, opp_stats1516)
team_a1617 = make_series(a, series_in1617, team_stats1617)
team_a_opp1617 = make_series(a, series_in1617, opp_stats1617)

team_a_opp1617.head()

,Unnamed: 0,Team,G,MP,FG,FGA,FG%,3P,3PA,3P.1,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
10,10,Boston_Celtics,82,240.9,38.6,85.8,0.450,9.0,27.0,0.332,...,0.769,10.8,33.8,44.5,22.6,7.6,5.2,13.9,20.3,105.4
13,13,Washington_Wizards,82,242.1,39.7,85.1,0.466,9.9,27.1,0.364,...,0.751,10.6,32.5,43.0,22.8,7.4,4.6,15.3,19.8,107.4
4,4,Toronto_Raptors,82,241.2,37.2,82.9,0.449,9.7,27.3,0.354,...,0.780,10.1,31.9,42.1,21.8,6.7,4.8,14.6,20.3,102.6
12,12,Cleveland_Cavaliers,82,242.4,40.6,88.8,0.458,10.4,28.7,0.361,...,0.773,11.0,33.2,44.2,24.7,7.3,4.3,12.6,20.6,107.2
7,7,Golden_State_Warriors,82,241.2,38.8,89.2,0.435,9.0,27.8,0.324,...,0.761,11.7,31.8,43.5,22.7,8.6,3.8,15.5,19.4,104.3


And now do the same process for all teams in team b for each year. Notice again that the teams line up correctly but on the right side, team b, in the series dataframe above. 

In [44]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM B'''
b = 'b'
team_b1314 = make_series(b, series_in1314, team_stats1314)
team_b_opp1314 = make_series(b, series_in1314, opp_stats1314)
team_b1415 = make_series(b, series_in1415, team_stats1415)
team_b_opp1415 = make_series(b, series_in1415, opp_stats1415)
team_b1516 = make_series(b, series_in1516, team_stats1516)
team_b_opp1516 = make_series(b, series_in1516, opp_stats1516)
team_b1617 = make_series(b, series_in1617, team_stats1617)
team_b_opp1617 = make_series(b, series_in1617, opp_stats1617)

team_b1617.head()

,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
13,13,15,Chicago_Bulls,82,241.2,38.6,87.1,0.444,7.6,22.3,...,12.2,34.1,46.3,22.6,7.8,4.8,13.6,17.7,102.9,50.0
12,12,11,Atlanta_Hawks,82,242.4,38.1,84.4,0.451,8.9,26.1,...,10.3,34.1,44.3,23.6,8.2,4.8,15.8,18.2,103.2,52.4
11,11,13,Milwaukee_Bucks,82,241.2,38.8,81.9,0.474,8.8,23.7,...,8.8,31.6,40.4,24.2,8.1,5.3,14.0,20.2,103.6,51.2
10,10,13,Indiana_Pacers,82,242.1,39.3,84.5,0.465,8.6,23.0,...,9.0,33.0,42.0,22.5,8.2,5.0,13.8,19.5,105.1,51.2
6,6,15,Portland_Trail_Blazers,82,243.0,39.5,86.1,0.459,10.4,27.7,...,10.1,33.5,43.7,21.1,7.0,5.0,13.7,21.2,107.9,50.0


Here we clean the winning data. We want to binarize the data for who won each game in the series. This will be used directly in the model to train it. 

In [45]:
'''THIS MAKES THE WINNER DATAFRAME'''
winner1314 = series_in1314['Winner']
winner1415 = series_in1415['Winner']
winner1516 = series_in1516['Winner']
winner1617 = series_in1617['Winner']

win1314 = pd.DataFrame(winner1314, columns = ["Winner"])
win1415 = pd.DataFrame(winner1415, columns = ["Winner"])
win1516 = pd.DataFrame(winner1516, columns = ["Winner"])
win1617 = pd.DataFrame(winner1617, columns = ["Winner"])

win1314.head()

,Winner
0,0
1,1
2,1
3,0
4,0


Here we want to remove a number of columns from the dataframes in l. 

In [46]:
l = [team_a_opp1314, team_a_opp1415, team_a_opp1516, team_a_opp1617, team_b_opp1314, team_b_opp1415, team_b_opp1516, team_b_opp1617]

for j in l:
    del j['Unnamed: 0']
    del j['Team']
    del j['G']
    del j['MP']

team_a_opp1314.columns

Index(['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

Here we create the columns for all of the opponent data. We plan to train the model with all the features in team a which includes team a stats for and team a stats against. Then we want to subtract the stats from team b, and train the algorithm with that data and the winner. 

In [47]:
opps = [team_a_opp1314, team_a_opp1415, team_a_opp1516, team_a_opp1617, team_b_opp1314, team_b_opp1415, team_b_opp1516, team_b_opp1617]

for i in opps:
    i.columns = ['FG_opp', 'FGA_opp', 'FG%_opp', '3P_opp', '3PA_opp', '3P%_opp', '2P_opp', '2PA_opp', '2P%_opp', 'FT_opp', 'FTA_opp',
       'FT%_opp', 'ORB_opp', 'DRB_opp', 'TRB_opp', 'AST_opp', 'STL_opp', 'BLK_opp', 'TOV_opp', 'PF_opp', 'PTS_opp']


Here we add the statistics against to the statistics for so that it is in one dataframe

In [48]:
opp_cols = team_a_opp1314.columns
opp_cols

# team_b1314[col] = list(team_b_opp1314[col])

for col in opp_cols:
    team_a1314[col] = list(team_a_opp1314[col])
    team_a1415[col] = list(team_a_opp1415[col])
    team_a1516[col] = list(team_a_opp1516[col])
    team_a1617[col] = list(team_a_opp1617[col])
    team_b1314[col] = list(team_b_opp1314[col])
    team_b1415[col] = list(team_b_opp1415[col])
    team_b1516[col] = list(team_b_opp1516[col])
    team_b1617[col] = list(team_b_opp1617[col])

team_a1617.head()

,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,FT%_opp,ORB_opp,DRB_opp,TRB_opp,AST_opp,STL_opp,BLK_opp,TOV_opp,PF_opp,PTS_opp
5,5,4,Boston_Celtics,82,240.9,38.6,85.1,0.454,12.0,33.4,...,0.769,10.8,33.8,44.5,22.6,7.6,5.2,13.9,20.3,105.4
3,3,9,Washington_Wizards,82,242.1,41.3,87.0,0.475,9.2,24.8,...,0.751,10.6,32.5,43.0,22.8,7.4,4.6,15.3,19.8,107.4
7,7,5,Toronto_Raptors,82,241.2,39.2,84.4,0.464,8.8,24.3,...,0.780,10.1,31.9,42.1,21.8,6.7,4.8,14.6,20.3,102.6
2,2,5,Cleveland_Cavaliers,82,242.4,39.9,84.9,0.470,13.0,33.9,...,0.773,11.0,33.2,44.2,24.7,7.3,4.3,12.6,20.6,107.2
0,0,1,Golden_State_Warriors,82,241.2,43.1,87.1,0.495,12.0,31.2,...,0.761,11.7,31.8,43.5,22.7,8.6,3.8,15.5,19.4,104.3


Oddly, we need to remove the unnamed column and team column once again. 

In [49]:
team_a = [team_a1314, team_a1415, team_a1516, team_a1617]
team_b = [team_b1314, team_b1415, team_b1516, team_b1617]
arr = team_a + team_b

for j in arr:
    del j['Unnamed: 0']
    del j['Team']
#     print(j.dtypes)
#     print(j)

Here we need the difference between team a and team b, and then store that into a new dataframe. This is used to train our model.

In [50]:
def find_diff(col, cols, year_a, year_b, df):
    col_name = cols[col]
    end = []
    diff_end = []
    diff_arr = list(year_a[col_name])
    diff_arr2 = list(year_b[col_name])
    for x in range(len(diff_arr)):
        diff = diff_arr[x] - diff_arr2[x]
        end.append(diff)
    df[col] = end

Here we find the difference and store it in the appropriate dataframe. 

In [51]:
cols = list(team_a1314.columns)
# print(len(cols))

df1314 = pd.DataFrame()
df1415 = pd.DataFrame()
df1516 = pd.DataFrame()
df1617 = pd.DataFrame()

years = [df1314, df1415, df1516, df1617]

for year in range(len(years)):
    for col in range(len(cols)):
        find_diff(col, cols, team_a[year], team_b[year], years[year])
df1516.shape

(15, 46)

Write the data to csv files to be used in our other notebook

In [52]:
df1314.to_csv('./preprocessed/final/diff1314.csv')
df1415.to_csv('./preprocessed/final/diff1415.csv')
df1516.to_csv('./preprocessed/final/diff1516.csv')
df1617.to_csv('./preprocessed/final/diff1617.csv')

Now we want to also normalize all of the data, so that we can run the model with normalized data as well. Here is the method.

In [53]:
def normalize_df(df):
    cols = df.columns
    for col in cols:
        df = df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    return df

Then normalize all of the data

In [54]:
df_norm1314 = normalize_df(df1314)
df_norm1415 = normalize_df(df1415)
df_norm1516 = normalize_df(df1516)
df_norm1617 = normalize_df(df1617)

Then save all of the normalized data to the appropriate csv file. 

In [55]:
df_norm1314.to_csv('./preprocessed/final/diff_norm1314.csv')
df_norm1415.to_csv('./preprocessed/final/diff_norm1415.csv')
df_norm1516.to_csv('./preprocessed/final/diff_norm1516.csv')
df_norm1617.to_csv('./preprocessed/final/diff_norm1617.csv')

The following code that has been commented out was used in part 2 of this project, but we did not use it in the running of our model. 

In [56]:
# '''WRITES TEAM A STATS TO CSV FILES'''
# team_a1314.to_csv('./preprocessed/final/playoffs_for_a_1314.csv')
# team_a1415.to_csv('./preprocessed/final/playoffs_for_a_1415.csv')
# team_a1516.to_csv('./preprocessed/final/playoffs_for_a_1516.csv')
# team_a1617.to_csv('./preprocessed/final/playoffs_for_a_1617.csv')

# team_a_opp1314.to_csv('./preprocessed/final/playoffs_opp_a_1314.csv')
# team_a_opp1415.to_csv('./preprocessed/final/playoffs_opp_a_1415.csv')
# team_a_opp1516.to_csv('./preprocessed/final/playoffs_opp_a_1516.csv')
# team_a_opp1617.to_csv('./preprocessed/final/playoffs_opp_a_1617.csv')

In [57]:
# '''WRITES TEAM B STATS TO CSV FILES'''
# team_b1314.to_csv('./preprocessed/final/playoffs_for_b_1314.csv')
# team_b1415.to_csv('./preprocessed/final/playoffs_for_b_1415.csv')
# team_b1516.to_csv('./preprocessed/final/playoffs_for_b_1516.csv')
# team_b1617.to_csv('./preprocessed/final/playoffs_for_b_1617.csv')

# team_b_opp1314.to_csv('./preprocessed/final/playoffs_opp_b_1314.csv')
# team_b_opp1415.to_csv('./preprocessed/final/playoffs_opp_b_1415.csv')
# team_b_opp1516.to_csv('./preprocessed/final/playoffs_opp_b_1516.csv')
# team_b_opp1617.to_csv('./preprocessed/final/playoffs_opp_b_1617.csv')

In [58]:
# '''WRITES WINNER DATAFRAME TO CSV FILES'''
#win1314.to_csv('./preprocessed/final/winner1314.csv')
#win1415.to_csv('./preprocessed/final/winner1415.csv')
#win1516.to_csv('./preprocessed/final/winner1516.csv')
#win1617.to_csv('./preprocessed/final/winner1617.csv')

In [59]:
#team_stats1718 drop unnamed, team, g, mp
#del team_stats1718['Unnamed: 0']
#del team_stats1718['Team']
#del team_stats1718['G']
#del team_stats1718['MP']

In [60]:
#opp_stats1718
#del opp_stats1718['Unnamed: 0']
#del opp_stats1718['Team']
#del opp_stats1718['G']
#del opp_stats1718['MP']

Here we prepare the data for this season, so that we can run it through our model and see who the winner is!

In [63]:
_columns = ['FG_opp', 'FGA_opp', 'FG%_opp', '3P_opp', '3PA_opp', '3P%_opp', '2P_opp', '2PA_opp', '2P%_opp', 'FT_opp', 'FTA_opp', 'FT%_opp', 'ORB_opp', 'DRB_opp', 'TRB_opp', 'AST_opp', 'STL_opp', 'BLK_opp', 'TOV_opp', 'PF_opp', 'PTS_opp']
_opp_cols = opp_stats1718.columns


for col in _opp_cols:
    team_stats1718[col] = list(opp_stats1718[col])
team_stats1718.columns

Index(['Unnamed: 0', 'Rk', 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'WIN%'],
      dtype='object')

In [25]:
opp_stats1718.head()

,Unnamed: 0,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,Utah_Jazz,82,240.9,37.3,83.0,0.449,9.5,26.0,0.365,...,0.771,8.7,33.0,41.6,20.0,8.3,4.6,14.9,20.5,99.8
1,1,San_Antonio_Spurs,82,240.6,38.2,84.3,0.453,9.1,26.3,0.348,...,0.759,9.2,33.3,42.5,21.8,7.6,3.9,14.1,19.8,99.8
2,2,Boston_Celtics,82,241.5,37.4,85.0,0.440,9.4,27.7,0.339,...,0.763,9.6,34.2,43.8,21.3,7.2,4.4,14.1,19.2,100.4
3,3,Miami_Heat,82,243.0,37.7,83.8,0.450,9.6,26.6,0.360,...,0.783,9.1,34.0,43.1,21.0,7.5,4.6,14.1,19.4,102.9
4,4,Portland_Trail_Blazers,82,240.9,38.4,85.9,0.447,9.7,26.6,0.364,...,0.755,9.3,33.6,42.9,20.2,7.3,5.1,12.6,19.1,103.0
